In [9]:
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversariantes():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta dos cartões)
    MES_PROCESSAR = 7  # Julho (mês do evento)
    ANO = datetime.now().year

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_IMAGEM_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "davi.araujo@grendene.com.br"

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA, header=0)
        df.columns = ['Nome Funcionário', 'Dia', 'Mês', 'MÊS_NUMERO']

        df_mes = df[df['MÊS_NUMERO'] == MES_CARDS]

        if df_mes.empty:
            print(f"Não há aniversariantes no mês {MES_CARDS}")
            return

        outlook = win32com.client.Dispatch("Outlook.Application")
        arquivos_cartoes = os.listdir(PASTA_CARDS)

        for dia, grupo in df_mes.groupby('Dia', sort=True):
            try:
                nomes = grupo['Nome Funcionário'].tolist()

                # Montar nomes em lista vertical com ";" no fim de cada linha (texto simples para evento)
                nomes_texto_lista = "\n".join(f"{nome};" for nome in nomes)

                print(f"\n[PROCESSANDO] Aniversariantes do dia {dia:02d}:\n{nomes_texto_lista}")

                data_evento = datetime(ANO, MES_PROCESSAR, int(dia), 8, 0)

                # === Criar Evento (texto simples, sem imagem da marca) ===
                appointment = outlook.CreateItem(1)  # olAppointmentItem

                if len(nomes) == 1:
                    titulo = f"Aniversário de {nomes[0]}"
                else:
                    titulo = f"Aniversário de {' e '.join(nomes)}"

                appointment.Subject = titulo
                appointment.Start = data_evento
                appointment.Duration = 30
                appointment.AllDayEvent = False
                appointment.ReminderSet = True
                appointment.ReminderMinutesBeforeStart = 1440
                appointment.BusyStatus = 0

                corpo_evento = f"""Olá!
Segue lembrete do aniversário para os colaboradores:
{nomes_texto_lista}

ATENÇÃO!: No dia do aniversário dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.
Atenciosamente,
"""

                appointment.Body = corpo_evento

                # Anexar cartões no evento
                anexados = 0
                for nome in nomes:
                    nome_lower = nome.lower()
                    encontrado = False
                    for arquivo in arquivos_cartoes:
                        if nome_lower in arquivo.lower():
                            caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                            appointment.Attachments.Add(caminho_cartao)
                            anexados += 1
                            encontrado = True
                            break
                    if not encontrado:
                        print(f"[AVISO] Cartão não encontrado para {nome} (evento)")

                print(f"[INFO] {anexados} cartões anexados ao evento.")

                appointment.Display()  # mostra para revisão antes de enviar

                # === Criar Email (HTML, com imagem da marca) ===
                mail = outlook.CreateItem(0)  # olMailItem
                mail.Subject = f"Lembrete de Aniversário - {dia:02d}/{MES_PROCESSAR:02d}"
                mail.To = EMAIL_DESTINO

                # Anexar cartões no email
                for nome in nomes:
                    nome_lower = nome.lower()
                    encontrado = False
                    for arquivo in arquivos_cartoes:
                        if nome_lower in arquivo.lower():
                            caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                            mail.Attachments.Add(caminho_cartao)
                            encontrado = True
                            break
                    if not encontrado:
                        print(f"[AVISO] Cartão não encontrado para {nome} (email)")

                # Adicionar imagem marca no email inline
                img_tag = ""
                if os.path.exists(CAMINHO_IMAGEM_MARCA):
                    attachment = mail.Attachments.Add(CAMINHO_IMAGEM_MARCA)
                    attachment.PropertyAccessor.SetProperty(
                        "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                    )
                    img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

                # Monta o texto dos nomes em HTML com <b> e <br> para cada linha
                nomes_html = "<br>".join(f"<b>{nome};</b>" for nome in nomes)

                corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete do aniversário para os colaboradores:<br>
{nomes_html}</p>

<p><b>ATENÇÃO!:</b> No dia do aniversário dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>

{img_tag}

</body>
</html>
"""

                mail.HTMLBody = corpo_email

                mail.Display()  # mostra para revisão antes de enviar

                print(f"[OK] Evento e email preparados para {nomes_texto_lista}")

            except Exception as e:
                print(f"[ERRO] Falha no dia {dia:02d}: {str(e)}")
                continue

        print("\n[OK] Processamento concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversariantes()



[PROCESSANDO] Aniversariantes do dia 01:
DENER DAVI SOUSA LIMA;
MARTIM LUIS CREMA;
[INFO] 2 cartões anexados ao evento.
[OK] Evento e email preparados para DENER DAVI SOUSA LIMA;
MARTIM LUIS CREMA;

[PROCESSANDO] Aniversariantes do dia 02:
AFRANIO BIN;
CAMILA DE AZEVEDO OLIVEIRA;
[INFO] 2 cartões anexados ao evento.
[OK] Evento e email preparados para AFRANIO BIN;
CAMILA DE AZEVEDO OLIVEIRA;

[PROCESSANDO] Aniversariantes do dia 04:
GABRIEL CAMARGO;
[INFO] 1 cartões anexados ao evento.
[OK] Evento e email preparados para GABRIEL CAMARGO;

[PROCESSANDO] Aniversariantes do dia 08:
JOHNATAS FELIX ZUZA;
PRISCILA MERZONI;
[INFO] 2 cartões anexados ao evento.
[OK] Evento e email preparados para JOHNATAS FELIX ZUZA;
PRISCILA MERZONI;

[PROCESSANDO] Aniversariantes do dia 11:
ELIZETE APARECIDA DE MATOS;
[INFO] 1 cartões anexados ao evento.
[OK] Evento e email preparados para ELIZETE APARECIDA DE MATOS;

[PROCESSANDO] Aniversariantes do dia 12:
NELSON DE ALMEIDA MACHADO BATISTA;
[INFO] 1 cartõ

In [ ]:
## Codigo teste
import os
import pandas as pd
import win32com.client
from datetime import datetime


def processar_aniversariantes_teste():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes.xlsx"
    NOME_ABA = "Sheet1"

    MES_CARDS = 3  # Março (pasta dos cartões)
    MES_PROCESSAR = 7  # Julho (mês do evento)
    ANO = datetime.now().year

    PASTA_CARDS = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Aniversariantes - Março"
    CAMINHO_IMAGEM_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\automation\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "davi.araujo@grendene.com.br"

    try:
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA, header=0)
        df.columns = ['Nome Funcionário', 'Dia', 'Mês', 'MÊS_NUMERO']

        df_mes = df[df['MÊS_NUMERO'] == MES_CARDS]

        if df_mes.empty:
            print(f"Não há aniversariantes no mês {MES_CARDS}")
            return

        outlook = win32com.client.Dispatch("Outlook.Application")
        arquivos_cartoes = os.listdir(PASTA_CARDS)

        # Limitar para os 2 primeiros aniversariantes para teste
        df_mes = df_mes.head(2)

        for dia, grupo in df_mes.groupby('Dia', sort=True):
            try:
                nomes = grupo['Nome Funcionário'].tolist()

                # Montar nomes em lista vertical com ";" no fim de cada linha
                nomes_texto_lista = "\n".join(f"{nome};" for nome in nomes)

                print(f"\n[PROCESSANDO] Aniversariantes do dia {dia:02d}:\n{nomes_texto_lista}")

                data_evento = datetime(ANO, MES_PROCESSAR, int(dia), 8, 0)

                # === Criar Evento (texto simples, sem imagem) ===
                appointment = outlook.CreateItem(1)  # olAppointmentItem

                if len(nomes) == 1:
                    titulo = f"Aniversário de {nomes[0]}"
                else:
                    titulo = f"Aniversário de {' e '.join(nomes)}"

                appointment.Subject = titulo
                appointment.Start = data_evento
                appointment.Duration = 30
                appointment.AllDayEvent = False
                appointment.ReminderSet = True
                appointment.ReminderMinutesBeforeStart = 1440
                appointment.BusyStatus = 0

                corpo_evento = f"""Olá!
Segue lembrete do aniversário para os colaboradores:
{nomes_texto_lista}

ATENÇÃO!: No dia do aniversário dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.
Atenciosamente,
"""

                appointment.Body = corpo_evento

                # Anexar cartões no evento
                anexados = 0
                for nome in nomes:
                    nome_lower = nome.lower()
                    encontrado = False
                    for arquivo in arquivos_cartoes:
                        if nome_lower in arquivo.lower():
                            caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                            appointment.Attachments.Add(caminho_cartao)
                            anexados += 1
                            encontrado = True
                            break
                    if not encontrado:
                        print(f"[AVISO] Cartão não encontrado para {nome} (evento)")

                print(f"[INFO] {anexados} cartões anexados ao evento.")

                appointment.Display()  # mostra para revisão

                # === Criar Email (HTML, com imagem da marca) ===
                mail = outlook.CreateItem(0)  # olMailItem
                mail.Subject = f"Lembrete de Aniversário - {dia:02d}/{MES_PROCESSAR:02d}"
                mail.To = EMAIL_DESTINO

                # Anexar cartões no email
                for nome in nomes:
                    nome_lower = nome.lower()
                    encontrado = False
                    for arquivo in arquivos_cartoes:
                        if nome_lower in arquivo.lower():
                            caminho_cartao = os.path.join(PASTA_CARDS, arquivo)
                            mail.Attachments.Add(caminho_cartao)
                            encontrado = True
                            break
                    if not encontrado:
                        print(f"[AVISO] Cartão não encontrado para {nome} (email)")

                # Adicionar imagem marca no email inline
                img_tag = ""
                if os.path.exists(CAMINHO_IMAGEM_MARCA):
                    attachment = mail.Attachments.Add(CAMINHO_IMAGEM_MARCA)
                    attachment.PropertyAccessor.SetProperty(
                        "http://schemas.microsoft.com/mapi/proptag/0x3712001F", "marca"
                    )
                    img_tag = '<p><img src="cid:marca" style="width:300px;"></p>'

                corpo_email = f"""
<html>
<body>
<p>Olá!</p>

<p>Segue lembrete do aniversário para os colaboradores:<br>
<pre style="font-family: Arial, sans-serif; white-space: pre-wrap;">{nomes_texto_lista}</pre></p>

<p><b>ATENÇÃO!:</b> No dia do aniversário dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>

<p>Atenciosamente,</p>

{img_tag}

</body>
</html>
"""

                mail.HTMLBody = corpo_email

                mail.Display()  # mostra para revisão

                print(f"[OK] Evento e email preparados para {nomes_texto_lista}")

            except Exception as e:
                print(f"[ERRO] Falha no dia {dia:02d}: {str(e)}")
                continue

        print("\n[OK] Teste concluído!")

    except Exception as e:
        print(f"[ERRO] Falha geral na automação: {str(e)}")


if __name__ == "__main__":
    processar_aniversariantes_teste()



[PROCESSANDO] Aniversariantes do dia 01:
DENER DAVI SOUSA LIMA;
MARTIM LUIS CREMA;
[INFO] 2 cartões anexados ao evento.
[OK] Evento e email preparados para DENER DAVI SOUSA LIMA;
MARTIM LUIS CREMA;

[OK] Teste concluído!
